# Assignment 2

In this assigment, we will work with the *Adult* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/2/adult). Extract the data files into the subdirectory: `../05_src/data/adult/` (relative to `./05_src/`).

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../05_src/data/adult/`, then you can use the code below to load them.

In [4]:
import pandas as pd
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
    'native-country', 'income'
]
adult_dt = (pd.read_csv('../../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))



   age          workclass  fnlwgt   education  education-num  \
0   39          State-gov   77516   Bachelors             13   
1   50   Self-emp-not-inc   83311   Bachelors             13   
2   38            Private  215646     HS-grad              9   
3   53            Private  234721        11th              7   
4   28            Private  338409   Bachelors             13   

        marital-status          occupation    relationship    race      sex  \
0        Never-married        Adm-clerical   Not-in-family   White     Male   
1   Married-civ-spouse     Exec-managerial         Husband   White     Male   
2             Divorced   Handlers-cleaners   Not-in-family   White     Male   
3   Married-civ-spouse   Handlers-cleaners         Husband   Black     Male   
4   Married-civ-spouse      Prof-specialty            Wife   Black   Female   

   capital-gain  capital-loss  hours-per-week  native-country  income  
0          2174             0              40   United-States       

# Get X and Y

Create the features data frame and target data:

+ Create a dataframe `X` that holds the features (all columns that are not `income`).
+ Create a dataframe `Y` that holds the target data (`income`).
+ From `X` and `Y`, obtain the training and testing data sets:

    - Use a train-test split of 70-30%. 
    - Set the random state of the splitting function to 42.

In [5]:
# Import train_test_split
from sklearn.model_selection import train_test_split 

# Create a DataFrame with feature variables
X = adult_dt.drop(columns='income')  
# Create a Series with the target variable
Y = adult_dt['income']  

# Split the data into training and testing sets with a 70-30% ratio and random stated of 42
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

# Display the shapes of the training and testing sets
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape


((22792, 14), (9769, 14), (22792,), (9769,))

## Random States

Please comment: 

+ What is the [random state](https://scikit-learn.org/stable/glossary.html#term-random_state) of the [splitting function](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)? 
+ Why is it [useful](https://en.wikipedia.org/wiki/Reproducibility)?

*(Comment here.)*
The random_state parameter sets a seed for the random number generator, ensuring that the data split is consistent each time the code runs. It is useful because it is important for reproducibility, as it allows for consistent results, making it easier to debug, compare models, and share findings with others.

# Preprocessing

Create a [Column Transformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) that treats the features as follows:

- Numerical variables

    * Apply [KNN-based imputation for completing missing values](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html):
        
        + Consider the 7 nearest neighbours.
        + Weight each neighbour by the inverse of its distance, causing closer neigbours to have more influence than more distant ones.
    * [Scale features using statistics that are robust to outliers](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html#sklearn.preprocessing.RobustScaler).

- Categorical variables: 
    
    * Apply a [simple imputation strategy](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer):

        + Use the most frequent value to complete missing values, also called the *mode*.

    * Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html):
        
        + Handle unknown labels if they exist.
        + Drop one column for binary variables.
    
    
The column transformer should look like this:

![](./images/assignment_2__column_transformer.png)

In [8]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn import set_config

# Set display configuration for pipelines
set_config(display='diagram')

# Define numerical and categorical feature lists
numerical_features = ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']
categorical_features = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']

# Define the preprocessing steps for numerical features
num_transforms = Pipeline(steps=[
    ('KNNImputer', KNNImputer(n_neighbors=7, weights='distance')),  # KNN imputer for handling missing values
    ('RobustScaler', RobustScaler())   # Scale features using RobustScaler to handle outliers
])

# Define the preprocessing steps for categorical features
cat_transforms = Pipeline(steps=[
    ('SimpleImputer', SimpleImputer(strategy='most_frequent')),  # Imputer that fills missing values with the most frequent value
    ('OneHotEncoder', OneHotEncoder(handle_unknown='ignore', drop='if_binary'))  # One-hot encoder that handles unknown categories
])

# Combine numerical and categorical pipelines using ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num_transforms', num_transforms, numerical_features),
        ('cat_transforms', cat_transforms, categorical_features)
    ]
)


# Apply the preprocessor to the dataset and display the shape of the transformed data
preprocessed_data = preprocessor.fit_transform(adult_dt)
print(f"Shape of preprocessed data: {preprocessed_data.shape}")
preprocessor

Shape of preprocessed data: (32561, 107)


ColumnTransformer(transformers=[('num_transforms',
                                 Pipeline(steps=[('KNNImputer',
                                                  KNNImputer(n_neighbors=7,
                                                             weights='distance')),
                                                 ('RobustScaler',
                                                  RobustScaler())]),
                                 ['age', 'fnlwgt', 'education-num',
                                  'capital-gain', 'capital-loss',
                                  'hours-per-week']),
                                ('cat_transforms',
                                 Pipeline(steps=[('SimpleImputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('OneHotEncoder',
                                                  OneHotEncoder(drop='if_binary',
                                                                handle_unknown='ignore'))]),
                                 ['workclass', 'education', 'marital-status',
                                  'occupation', 'relationship', 'race', 'sex',
                                  'native-country'])])

## Model Pipeline

Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `classifier` and assign a [`RandomForestClassifier()`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) to it.

The pipeline looks like this:

![](./images/assignment_2__pipeline.png)

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

# Construct the model pipeline with a preprocessor and RandomForest classifier
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),  # Use the ColumnTransformer from the previous section
    ('random_forest', RandomForestClassifier(random_state=42))  # RandomForestClassifier with a set random state for reproducibility
])

# Display the pipeline to confirm its structure
model_pipeline


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num_transforms',
                                                  Pipeline(steps=[('KNNImputer',
                                                                   KNNImputer(n_neighbors=7,
                                                                              weights='distance')),
                                                                  ('RobustScaler',
                                                                   RobustScaler())]),
                                                  ['age', 'fnlwgt',
                                                   'education-num',
                                                   'capital-gain',
                                                   'capital-loss',
                                                   'hours-per-week']),
                                                 ('cat_transforms',
                                                  Pipeline(steps=[('SimpleImputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('OneHotEncoder',
                                                                   OneHotEncoder(drop='if_binary',
                                                                                 handle_unknown='ignore'))]),
                                                  ['workclass', 'education',
                                                   'marital-status',
                                                   'occupation', 'relationship',
                                                   'race', 'sex',
                                                   'native-country'])])),
                ('random_forest', RandomForestClassifier(random_state=42))])

# Cross-Validation

Evaluate the model pipeline using [`cross_validate()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html):

+ Measure the following [preformance metrics](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values): negative log loss, ROC AUC, accuracy, and balanced accuracy.
+ Report the training and validation results. 
+ Use five folds.


In [10]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer, log_loss, roc_auc_score, accuracy_score, balanced_accuracy_score

# Define the scoring metrics
scoring_metrics = {
    'neg_log_loss': make_scorer(log_loss, needs_proba=True),  # Negative log loss
    'roc_auc': make_scorer(roc_auc_score),  # ROC AUC score
    'accuracy': make_scorer(accuracy_score),  # Accuracy score
    'balanced_accuracy': make_scorer(balanced_accuracy_score)  # Balanced accuracy score
}

# Performs cross validation using 5 folds
cv_results = cross_validate(
    model_pipeline, adult_dt, Y, 
    cv=5, 
    scoring=scoring_metrics, 
    return_train_score=True
)

# Display the mean training and validation scores for each metric
print("Training and Validation Results:")
print(f"Training Negative Log Loss: {cv_results['train_neg_log_loss'].mean():.4f}")
print(f"Validation Negative Log Loss: {cv_results['test_neg_log_loss'].mean():.4f}")
print(f"Training ROC AUC: {cv_results['train_roc_auc'].mean():.4f}")
print(f"Validation ROC AUC: {cv_results['test_roc_auc'].mean():.4f}")
print(f"Training Accuracy: {cv_results['train_accuracy'].mean():.4f}")
print(f"Validation Accuracy: {cv_results['test_accuracy'].mean():.4f}")
print(f"Training Balanced Accuracy: {cv_results['train_balanced_accuracy'].mean():.4f}")
print(f"Validation Balanced Accuracy: {cv_results['test_balanced_accuracy'].mean():.4f}")

/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/metrics/_scorer.py:610: FutureWarning: The `needs_threshold` and `needs_proba` parameter are deprecated in version 1.4 and will be removed in 1.6. You can either let `response_method` be `None` or set it to `predict` to preserve the same behaviour.
  warnings.warn(
/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


Training and Validation Results:
Training Negative Log Loss: 0.0805
Validation Negative Log Loss: 0.3738
Training ROC AUC: 1.0000
Validation ROC AUC: 0.7748
Training Accuracy: 1.0000
Validation Accuracy: 0.8542
Training Balanced Accuracy: 1.0000
Validation Balanced Accuracy: 0.7748


Display the fold-level results as a pandas data frame and sorted by negative log loss of the test (validation) set.

In [11]:
import pandas as pd

# Create a DataFrame from the cross-validation results
results_df = pd.DataFrame({
    'fold': range(1, len(cv_results['train_neg_log_loss']) + 1),  # Automatically adjust the number of folds
    'train_neg_log_loss': cv_results['train_neg_log_loss'],
    'test_neg_log_loss': cv_results['test_neg_log_loss'],
    'train_roc_auc': cv_results['train_roc_auc'],
    'test_roc_auc': cv_results['test_roc_auc'],
    'train_accuracy': cv_results['train_accuracy'],
    'test_accuracy': cv_results['test_accuracy'],
    'train_balanced_accuracy': cv_results['train_balanced_accuracy'],
    'test_balanced_accuracy': cv_results['test_balanced_accuracy']
})

# Sort the DataFrame by test negative log loss in ascending order
sorted_results_df = results_df.sort_values(by='test_neg_log_loss', ascending=True)

# Display the sorted DataFrame
print(sorted_results_df)


   fold  train_neg_log_loss  test_neg_log_loss  train_roc_auc  test_roc_auc  \
2     3            0.080541           0.359862       0.999975      0.779941   
3     4            0.080482           0.362359       1.000000      0.774328   
4     5            0.081150           0.371268       0.999975      0.782874   
0     1            0.080316           0.374309       0.999920      0.770399   
1     2            0.079911           0.401078       0.999975      0.766238   

   train_accuracy  test_accuracy  train_balanced_accuracy  \
2        0.999962       0.853655                 0.999975   
3        1.000000       0.857033                 1.000000   
4        0.999962       0.858108                 0.999975   
0        0.999962       0.853217                 0.999920   
1        0.999962       0.849048                 0.999975   

   test_balanced_accuracy  
2                0.779941  
3                0.774328  
4                0.782874  
0                0.770399  
1                0

Calculate the mean of each metric. 

In [12]:
import pandas as pd

# Calculate the mean performance metrics
mean_metrics = {
    'Mean Train Negative Log Loss': -results_df['train_neg_log_loss'].mean(),
    'Mean Test Negative Log Loss': -results_df['test_neg_log_loss'].mean(),
    'Mean Train ROC AUC': results_df['train_roc_auc'].mean(),
    'Mean Test ROC AUC': results_df['test_roc_auc'].mean(),
    'Mean Train Accuracy': results_df['train_accuracy'].mean(),
    'Mean Test Accuracy': results_df['test_accuracy'].mean(),
    'Mean Train Balanced Accuracy': results_df['train_balanced_accuracy'].mean(),
    'Mean Test Balanced Accuracy': results_df['test_balanced_accuracy'].mean()
}

# Convert the mean metrics to a DataFrame for better visualization
mean_metrics_df = pd.DataFrame([mean_metrics])

# Display the mean metrics in a cleaner format
print("Mean Performance Metrics:")
print(mean_metrics_df.to_string(index=False))


Mean Performance Metrics:
 Mean Train Negative Log Loss  Mean Test Negative Log Loss  Mean Train ROC AUC  Mean Test ROC AUC  Mean Train Accuracy  Mean Test Accuracy  Mean Train Balanced Accuracy  Mean Test Balanced Accuracy
                     -0.08048                    -0.373776            0.999969           0.774756             0.999969            0.854212                      0.999969                     0.774756


Calculate the same performance metrics (negative log loss, ROC AUC, accuracy, and balanced accuracy) using the testing data `X_test` and `Y_test`. Display results as a dictionary.

*Tip*: both, `roc_auc()` and `neg_log_loss()` will require prediction scores from `pipe.predict_proba()`. However, for `roc_auc()` you should only pass the last column `Y_pred_proba[:, 1]`. Use `Y_pred_proba` with `neg_log_loss()`.

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss, roc_auc_score, accuracy_score, balanced_accuracy_score

# Split the data into training and testing sets if not already done
X_train, X_test, Y_train, Y_test = train_test_split(adult_dt, Y, test_size=0.3, random_state=42)

# Fit the model pipeline to the training data
model_pipeline.fit(X_train, Y_train)

# Make predictions and obtain prediction probabilities
Y_pred = model_pipeline.predict(X_test)
Y_pred_proba = model_pipeline.predict_proba(X_test)

# Calculate performance metrics using the testing data
performance_metrics = {
    'Negative Log Loss': log_loss(Y_test, Y_pred_proba),  # Log loss (lower is better)
    'ROC AUC': roc_auc_score(Y_test, Y_pred_proba[:, 1]),  # Use positive class probabilities for ROC AUC
    'Accuracy': accuracy_score(Y_test, Y_pred),  # Accuracy (higher is better)
    'Balanced Accuracy': balanced_accuracy_score(Y_test, Y_pred)  # Balanced accuracy for imbalanced classes
}

# Display the performance metrics as a dictionary
print("Performance Metrics:")
for metric, value in performance_metrics.items():
    print(f"{metric}: {value:.4f}")


Performance Metrics:
Negative Log Loss: 0.4007
ROC AUC: 0.8997
Accuracy: 0.8543
Balanced Accuracy: 0.7748


# Target Recoding

In the first code chunk of this document, we loaded the data and immediately recoded the target variable `income`. Why is this [convenient](https://scikit-learn.org/stable/modules/model_evaluation.html#binary-case)?

The specific line was:

```
adult_dt = (pd.read_csv('../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))
```

(Answer here.)
Recoding income immediately simplifies data preparation and ensures consistency. For ROC AUC calculation, it is essential that the label of interest (e.g., >50K) is encoded as 1, as the metric evaluates the model's ability to distinguish between positive (1) and negative (0) classes.

## Criteria

The [rubric](./assignment_2_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Becker,Barry and Kohavi,Ronny. (1996). Adult. UCI Machine Learning Repository. https://doi.org/10.24432/C5XW20.